## 1 Normalizers

In [27]:
from tokenizers.normalizers import Lowercase, NFC, BertNormalizer

In [33]:
#Sample text
text = "EsTá ORÄCIÖN Sé DèBÈ nOŔmALÏZÁR"
chinese_text = "COŃ CARáctEŔès chINös 手田水口廿卜山"

In [34]:
#Create normalizers objects
lowercase_norm = Lowercase()
nfc_norm = NFC()
bert_norm = BertNormalizer()

In [35]:
print(f"Lowercase normalizer:\n{lowercase_norm.normalize_str(text)}")
print(f"NFC normalizer:\n{nfc_norm.normalize_str(text)}")
print(f"BERT normalizer:\n{bert_norm.normalize_str(text)}")

Lowercase normalizer:
está oräciön sé dèbè noŕmalïzár
NFC normalizer:
EsTá ORÄCIÖN Sé DèBÈ nOŔmALÏZÁR
BERT normalizer:
esta oracion se debe normalizar


In [36]:
print(f"BERT normalizer with chinese chars:\n{bert_norm.normalize_str(chinese_text)}")

BERT normalizer with chinese chars:
con caracteres chinos  手  田  水  口  廿  卜  山 


## 2 Pre-Tokenization

In [32]:
from tokenizers.pre_tokenizers import WhitespaceSplit, BertPreTokenizer

In [51]:
text = ("Este texto contiene comas, puntos y comas; parentesís () " \
        "y puntos. También apóstrofes Mickey's")

In [52]:
#Create pre-tokenizer objects
whitespace_split = WhitespaceSplit()
bert_pretokenizer = BertPreTokenizer()

In [53]:
print("Whitespace split pretokenizer:")
whitespace_split.pre_tokenize_str(text)

Whitespace split pretokenizer:


[('Este', (0, 4)),
 ('texto', (5, 10)),
 ('contiene', (11, 19)),
 ('comas,', (20, 26)),
 ('puntos', (27, 33)),
 ('y', (34, 35)),
 ('comas;', (36, 42)),
 ('parentesís', (43, 53)),
 ('()', (54, 56)),
 ('y', (57, 58)),
 ('puntos.', (59, 66)),
 ('También', (67, 74)),
 ('apóstrofes', (75, 85)),
 ("Mickey's", (86, 94))]

Nota: La tupla que se genera al lado del token corresponde a las posiciones de inicio y final (no inclusivo) del token dentro del texto fuente

In [54]:
print("Bert pre tokenizer:")
bert_pretokenizer.pre_tokenize_str(text)

Bert pre tokenizer:


[('Este', (0, 4)),
 ('texto', (5, 10)),
 ('contiene', (11, 19)),
 ('comas', (20, 25)),
 (',', (25, 26)),
 ('puntos', (27, 33)),
 ('y', (34, 35)),
 ('comas', (36, 41)),
 (';', (41, 42)),
 ('parentesís', (43, 53)),
 ('(', (54, 55)),
 (')', (55, 56)),
 ('y', (57, 58)),
 ('puntos', (59, 65)),
 ('.', (65, 66)),
 ('También', (67, 74)),
 ('apóstrofes', (75, 85)),
 ('Mickey', (86, 92)),
 ("'", (92, 93)),
 ('s', (93, 94))]

In [55]:
from torch import * #Transformers has dependencies with pytorch
from transformers import AutoTokenizer

In [48]:
#Pre-tokenizers from common models: GPT2 and ALBERT (A LiteBert)
GPT2_pretokenizer = AutoTokenizer.from_pretrained('gpt2').backend_tokenizer.pre_tokenizer
ALBERT_pretokenizer = AutoTokenizer.from_pretrained('albert-base-v1').backend_tokenizer.pre_tokenizer

In [56]:
print('GPT2 pre tokenizer:')
GPT2_pretokenizer.pre_tokenize_str(text)

GPT2 pre tokenizer:


[('Este', (0, 4)),
 ('Ġtexto', (4, 10)),
 ('Ġcontiene', (10, 19)),
 ('Ġcomas', (19, 25)),
 (',', (25, 26)),
 ('Ġpuntos', (26, 33)),
 ('Ġy', (33, 35)),
 ('Ġcomas', (35, 41)),
 (';', (41, 42)),
 ('ĠparentesÃŃs', (42, 53)),
 ('Ġ()', (53, 56)),
 ('Ġy', (56, 58)),
 ('Ġpuntos', (58, 65)),
 ('.', (65, 66)),
 ('ĠTambiÃ©n', (66, 74)),
 ('ĠapÃ³strofes', (74, 85)),
 ('ĠMickey', (85, 92)),
 ("'s", (92, 94))]

Nota: GPT2 representa los whitespaces con una variante de G especial

In [57]:
print('ALBERT pre tokenizer:')
ALBERT_pretokenizer.pre_tokenize_str(text)

ALBERT pre tokenizer:


[('▁Este', (0, 4)),
 ('▁texto', (5, 10)),
 ('▁contiene', (11, 19)),
 ('▁comas,', (20, 26)),
 ('▁puntos', (27, 33)),
 ('▁y', (34, 35)),
 ('▁comas;', (36, 42)),
 ('▁parentesís', (43, 53)),
 ('▁()', (54, 56)),
 ('▁y', (57, 58)),
 ('▁puntos.', (59, 66)),
 ('▁También', (67, 74)),
 ('▁apóstrofes', (75, 85)),
 ("▁Mickey's", (86, 94))]

Nota: Bert representa los whitespaces con guión bajo _

## 3 Tokenization

In [59]:
from tokenizers import Tokenizer

In [70]:
text = "Texto normalizado con el que podemos trabajar"

In [85]:
tokenizer = Tokenizer.from_pretrained('bert-base-uncased')
tokenizer.post_processor = None #"Disable" post-processing

In [86]:
output = tokenizer.encode(text)
print("Tokens: ")
list(zip(output.tokens, output.ids))

Tokens: 


[('text', 3793),
 ('##o', 2080),
 ('normal', 3671),
 ('##iza', 21335),
 ('##do', 3527),
 ('con', 9530),
 ('el', 3449),
 ('que', 10861),
 ('pod', 17491),
 ('##em', 6633),
 ('##os', 2891),
 ('tr', 19817),
 ('##aba', 19736),
 ('##jar', 16084)]

## 4 Post-processing

In [80]:
from tokenizers.processors import BertProcessing

In [81]:
bert_processor = BertProcessing(sep = ("[SEP]", 1), cls =("[CLS]", 2))

tokenizer = Tokenizer.from_pretrained('bert-base-uncased')
tokenizer.post_processor = bert_processor

Nota: Bert agrega los tokens [SEP] y [CLS] como parte de su post-procesado

In [84]:
output = tokenizer.encode(text)

print("Tokens: ")
list(zip(output.tokens, output.ids))

Tokens: 


[('[CLS]', 2),
 ('text', 3793),
 ('##o', 2080),
 ('normal', 3671),
 ('##iza', 21335),
 ('##do', 3527),
 ('con', 9530),
 ('el', 3449),
 ('que', 10861),
 ('pod', 17491),
 ('##em', 6633),
 ('##os', 2891),
 ('tr', 19817),
 ('##aba', 19736),
 ('##jar', 16084),
 ('[SEP]', 1)]